Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment) Jonatan Rivera

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [24]:
# Import Statements
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

### Load Competition Data

In [25]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [26]:
#Make a brief exploratory analysis
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4087 entries, 0 to 4086
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              4087 non-null   int64 
 1   description     4087 non-null   object
 2   ratingCategory  4087 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 95.9+ KB


In [67]:
#load nlp model in spacy
import spacy
nlp = spacy.load("en_core_web_lg")


### Define Pipeline Components

In [68]:
#Insert TfidVectorizer and RandomForestClassifier Into one Pipeline
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [59]:
#Perform a grid search cv , based on Tfidf vectorize and randoforesstclassifier Pipeline
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators':(150,)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [60]:
#Get accuracy score
grid_search.best_score_

0.7051632036821456

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [62]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [63]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [64]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [65]:
subNumber = 0

In [66]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


In [38]:

from sklearn.decomposition import TruncatedSVD


### Define Pipeline Components

In [39]:
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [49]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)

}


grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)


In [50]:
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed: 10.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

### Make a Submission File

In [83]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [84]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [85]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [86]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

In [39]:
#prediction vector
y = train['ratingCategory']
#import train validation split so that we can have a validation set to compare accuracy against
from sklearn.model_selection import train_test_split
#Perform Train Validation Split
X_train, X_val, y_train, y_val = train_test_split(train['description'], y, test_size=0.33, random_state=42)


## Follow Along

In [106]:

#import Randomized Search CV, to be used to help with Randomized search on hyper parameters.
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

#instantiate random forest classifier
rfc = RandomForestClassifier()
#define pipeline components
pipe = Pipeline([
                 # Classifier
                 ('clf', rfc)
                ])
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
params = { 'clf__n_estimators':(150,400,),
          'clf__max_depth':(2,10,15,20,),
          'clf__min_samples_leaf':(2,4,6,10),
          'clf__min_samples_split':(2,4,6)} 

In [81]:
type(get_lemmas)

function

In [76]:
#Get word vectors for data
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

#Apply grid search on train description(data) and use param from above
train_vectorized = get_word_vectors(X_train)



In [107]:
random_search = RandomizedSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
random_search.fit(train_vectorized, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('clf',
                                              RandomForestClassifier(bootstrap=True,
                                                                     ccp_alpha=0.0,
                                                                     class_weight=None,
                                                                     criterion='gini',
                                                                     max_depth=None,
                                                                     max_features='auto',
                                                                     max_leaf_nodes=None,
                                                                     max_samples=None,
                                                                     min_impurity_decrease=0.0,
                                                              

In [108]:
#Training Score
random_search.best_score_

0.7370361227131401

In [109]:
#Get vectors for the val description column
val_vectorized = get_word_vectors(X_val)

#Check Validation Score
val_pred = random_search.predict(val_vectorized)

In [110]:
#import accuracy score to check how accurace my val_pred is
from sklearn.metrics import accuracy_score
accuracy_score(y_val, val_pred)

0.7353595255744997

In [ ]:
#History Information
# For a RandomizedSearchCVfrom with parameters {'clf__n_estimators':(100,150),'clf__max_depth':(2,10),    'clf__min_samples_leaf':(2,4,6)},  y_val score is 73.3 percent
# For a RandomSearchCV w/ params = { 'clf__n_estimators':(300,), 'clf__max_depth':(2,10),'clf__min_samples_leaf':(2,4,6,10)} y_val is 72.5%
# For a RCV w/ params = { 'clf__n_estimators':(500,),'clf__max_depth':(2,10),'clf__min_samples_leaf':(2,4,6,10)} y/val is 73.09%
# For a RCV w/ params = { 'clf__n_estimators':(1000,),'clf__max_depth':(2,10),'clf__min_samples_leaf':(2,4,6,10)} y_val is 72.87%
# For a RCV w/ Params = params = { 'clf__n_estimators':(150,400,),'clf__max_depth':(2,10,15,20,),'clf__min_samples_leaf':(2,4,6,10),'clf__min_samples_split':(2,4,6)} y_val is 0.7346182357301705
# For a RCV w/ Params params = { 'clf__n_estimators':(400,1500),'clf__max_depth':(2,10,15,20,30),'clf__min_samples_leaf':(2,4,6,10),'clf__min_samples_split':(2,4,6)} y_val is 73.53%


### Make a Submission File

In [111]:
#vectorized test set and get predictions
test_vectorized = get_word_vectors(test['description'])

In [112]:
# Predictions on test sample
pred = random_search.predict(test_vectorized)


In [113]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [116]:
# Make Sure the Category is an Integer http://localhost:8888/?token=f5898eafed4c045964809fa47adaf7e363a3b67fad6be7d7

submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [115]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
  - What is "Sentiment Analysis"? 
    - Sentiment analysis is the interpretation and classification of emotions (positive, negative and neutral) within text data using text analysis techniques. Sentiment analysis tools allow       businesses to identify customer sentiment toward products, brands or services in online feedback.
    https://monkeylearn.com/sentiment-analysis


  - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
      - Sentiment analysis is also a type of Text classification, which involves classifying text by performing specific techniques on your text-based documents.
      - Document classification, however is the act of labeling documents into categories according to their content. Document classification can be manual (as it is in library science) or   automated (within the field of computer science), and is used to easily sort and manage texts, images or videos.
    https://monkeylearn.com/blog/document-classification/
 
- How do u create labeled sentiment data? Are those labels really sentiment?
    - Create a list of emoticons having positive sentiment and another list for negative sentiments. Then if text contain only(or mostly) emoticons of positive sentiment then label it as positive tweet and vice verse for negative label.
    - Use unsupervised algorithms to classify them, for them you need to choose your features like containing words, number of stopwords etc.
    - Use sentiwordnet to get a sentiment of overall tweet through sense of each word. You need to carefully model this.
    - Another approach would be find to Key aspects of a brand’s product and service that customers care about and look for Users’ underlying intentions and reactions concerning those aspects.
 
 https://www.quora.com/How-do-I-label-huge-Twitter-data-set-for-training-a-sentiment-analysis-classifier-without-manually-labeling-them
- What are common applications of sentiment analysis?
    - Social media monitoring
    - Customer support
    - Customer feedback
    - Brand monitoring
    - Voice of customer (VoC)
    - Voice of employee
    - Product analysis
    - Market research and competitive research
    https://monkeylearn.com/blog/sentiment-analysis-applications/#:~:text=Sentiment%20analysis%20uses%20machine%20learning,positive%2C%20negative%2C%20or%20neutral.
 

2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case? Neural Networks are designed to learn from numerical data. Word Embedding is really all about improving the ability of networks to learn from text data. By representing that data as lower dimensional vectors. These vectors are called Embedding. This technique is used to reduce the dimensionality of text data but these models can also learn some interesting traits about words in a vocabulary